In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
# Libraries
import os
import pickle

from src.models import train_model, test_model

In [18]:
# os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_pi/')
os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_pi/')

In [19]:
EXPERIMENT = 'mh06'
MODEL = 'model_A1'
EXP_DIR = f'./models/{EXPERIMENT}/'
# Training
PATH_TO_DATA = f'data/processed/synthetic_data/{MODEL}/n1000/'
# Models
model_template = f'./models/templates/{MODEL}.pl'

In [20]:
fractions = [0.1, 1, 5, 10, 20]
mutate = ['m1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7']

In [ ]:
for frac in fractions:
    for m in mutate:
        path_to_data = f'{PATH_TO_DATA}missing_{m}_{frac}/'
        train_model.main(
            experiment = EXPERIMENT, 
            path_to_data = path_to_data, 
            model_template = model_template, 
            training = 0.8, 
            max_iter = 3, 
            evidence = ['activity', 'occupancy', 'fold_change'],
            facts = [],
            suffix = f'_{m}_{frac}'
        )

In [21]:
# Testing
test = ['training', 'testing']
PATH_TO_TRUTH = f'data/processed/synthetic_data/{MODEL}/n1000/'
results = {}

In [ ]:
for frac in fractions:
    for m in mutate:
        model_trained = f'{EXP_DIR}model_trained_{m}_{frac}.pl'
        results[f'{m}_{frac}'] = {}
        for test_on in test:
            PATH_TO_TESTING = f'{EXP_DIR}{test_on}_{m}_{frac}/'
            compare_to = [test_on, 'truth']
            for compare in compare_to:
                if compare == 'truth':
                    path_to_truth = PATH_TO_TRUTH
                else: 
                    path_to_truth = PATH_TO_TESTING

                test_results = test_model.main(
                    experiment = EXPERIMENT, 
                    path_to_data = PATH_TO_TESTING, 
                    path_to_truth = path_to_truth, 
                    model_trained = model_trained, 
                    targets = ['occupancy', 'modification'], 
                    test_on = f'{test_on}_{m}_{frac}', compare = f'{compare}_{m}_{frac}')
                results[f'{m}_{frac}'][f'{test_on}_vs_{compare}'] = test_results

In [23]:
with open(f'{EXP_DIR}results.pickle', 'wb') as file:
    pickle.dump(results, file, protocol=pickle.HIGHEST_PROTOCOL)